In [1]:
import os
import pandas as pd
import numpy as np
import re

# llm
from dotenv import load_dotenv
from openai import OpenAI
import json

# db
import psycopg2

# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [2]:
# .env 파일 로드
load_dotenv()  # 현재 디렉토리 내 .env 파일 정보를 환경변수로 읽어옴

# 환경변수 확인
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found. .env 또는 시스템 환경변수에 설정하세요.")

client = OpenAI(api_key=api_key)
llm_model = "gpt-4o-mini"
embed_model = "text-embedding-ada-002"

### **[기존 함수]**

In [11]:
def value_to_strCSV(value):
    """
    리스트 또는 쉼표 문자열을 'a, b, c' 형태의 문자열로 표준화
    """
    if not value:
        return ""

    # 이미 리스트이면 → 요소 strip 후 join
    if isinstance(value, list):
        return ", ".join(x.strip() for x in value)

    # 문자열이면 split → 다시 join 처리
    if isinstance(value, str):
        return ", ".join(x.strip() for x in value.split(","))

    # 그 외 타입은 문자열로 강제 변환
    return str(value)

In [ ]:
def get_article_summary(title, contents, publish_date, model="gpt-4o-mini"):
    """
    뉴스 기사를 LLM으로 요약하고 항목별 데이터 반환
    
    Change Log:
        @@ 11.18 
        > 프롬프트 수정
            - 키값 정리 형식 재 설정 - 데이터 프레임의 컬럼명이 키값이 되도록 수정
            - 추가적인 컬럼 event_person 추가
            - event_obj 컬럼명 event_org로 변경
        @@ 11.19
        > 프롬프트 수정
            - 인물명은 이름만 명확하게
            - 지역명은 [국가, 도, 시] 단위로 명확하게
            - 평양 쌀, 옥수수, 달러환율 정보는 각기 별개의 데이터로 저장
            - 키워드는 인물명, 지역명을 반드시 포함 + 요약내용 대표 단어 추가
        > result 파싱 방식 수정
            - eval()에서 jason.load()로 변경
    """
    
    prompt = f"""
    아래 기사를 분석하여 요구된 정보를 작성하시오.

    # 기사 제목:
    {title}

    # 기사 내용:
    {contents}

    # 기사 작성일:
    {publish_date}

   1. 아래 형식으로 정리 (괄호안 각 key값의 한글 설명은 참고만 하고 최종 결과에는 포함하지 않음)
    - summary(주요 사건 요약):
    - event_title(사건 주제):
    - event_date(사건 발생일):
    - event_person(사건 핵심 인물):
    - event_org(사건 핵심 조직/기관):
    - event_loc(사건 발생 지명):
    - keywords(주요 키워드):
    - p_rice(won/kg)(쌀 가격):
    - p_corn(won/kg)(옥수수 가격):
    - p_usd(won/usd)(달러 가격):
    
    2. 각 카테고리의 조건
    - "summary": 3 문장 이하로 핵심 내용만 발췌.
    - "event_title": 간단한 한 문장으로 사건 주제 작성.
    - "event_date": yyyy-mm-dd 형식, 기사에 "event_date"가 명시되지 않았으면 "기사 내용" 중 시간 또는 기간을 나타내는 단어(예시로, '어제', '사흘전', '일주일 전' 등)를 참고하여 "기사 작성일" 기준 계산.
    - "event_person": 사건의 주체 인물(들)의 이름만 입력, 다수의 경우 쉼표로 구분.
    - "event_org": 사건의 주체 조직 및 기관의 이름만 입력, 다수의 경우 쉼표로 구분, **언론사명은 반드시 제외**, **신문사명은 반드시 제외**, **기자가 참고한 출처의 이름도 반드시 제외**, **"노동신문"은 반드시 제외**.
    - "event_loc": [도, 시]단위 지명만을 입력하되 "도" 와 "시" 정보가 함께 있는 경우는 반드시 행적구역별로 분리해서 입력. 건물등에서 일어난 사건의 경우는 해당 장소의 [도, 시] 지명을 입력, 행정구역이 "시"일 경우는 꼭 "시"를 명시 (개성시, 평양시, 고성시 등). 
    특히 "평양" / "평양직할시" / "평양시"와 같이 한 지명에 다양한 표기가 있을경우는 "평양시" ([시 이름] + 시)와 같은 형태로 통일. **"북한" 이라는 단어는 반드시 제외**. 북한이 아닌 해외의 사건의 경우만 국가명을 입력.
    - "keywords": "summary", "event_title", "event_person", "event_org", "event_loc" 모두를 종합적으로 고려하여 해당 뉴스 사건을 대표할 수 있는 **단어 5개 선정**, **"북한" 이라는 단어는 반드시 제외**, 쉼표로 구분하여 입력.
    - p_rice(won/kg): 반드시 **평양의 "쌀 가격"**이 확실하게 명시되어있는 경우만 추출, **숫자만** 입력. "평양"의 물가 정보만을 추출하기 위함. "평양"의 쌀 가격이라는 근거가 100% 명확하지 않은 경우 **절대로 지어내지 말고 빈칸**으로 남김. 
    - p_corn(won/kg): 반드시 **평양의 "옥수수가격"**이 확실하게 명시되어있는 경우만 추출, **숫자만** 입력. "평양"의 물가 정보만을 추출하기 위함. "평양"의 옥수수 가격이라는 근거가 100% 명확하지 않은 경우 **절대로 지어내지 말고 빈칸**으로 남김. 
    - p_usd(won/usd): 반드시 **평양의 "달러 가격"**이 확실하게 명시되어있는 경우만 추출, **숫자만** 입력. "평양"의 물가 정보만을 추출하기 위함. "평양"의 달러 환율이라는 근거가 100% 명확하지 않은 경우 **절대로 지어내지 말고 빈칸**으로 남김.
    
    - 위 결과를 종합하여 딕셔너리 형태로 출력.
    - 결과를 출력하기 전 다음 체크리스트를 스스로 검증하라:
        - [ ] 내가 사용한 모든 답과 수치는 기사 원문에 존재한다.
        - [ ] 서로 다른 가격(예: 식용유 vs 쌀 등)을 혼동하지 않았다.
        
    - 설명 출력 금지, 답만 출력.
    """
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": "당신은 북한 관련 뉴스 사건 정보를 추출하는 전문 분석 모델입니다."},
            {"role": "user", "content": prompt},
        ],
        temperature=0
    )
    
    # 소모 토큰량 추출
    # input_tokens = response.usage.prompt_tokens
    # output_tokens = response.usage.completion_tokens

    # LLM 결과 출력 가져오기
    result_text = response.choices[0].message.content.strip()

    # 문자열을 dict로 변환
    try:
        result = json.loads(result_text)  # json 파싱으로 수정(11.19)
    except:
        print("Parsing error:", result_text)
        return None

    # return result, input_tokens, output_tokens
    return result

In [13]:
def get_summary_df(df, cache_file="cache_output.csv", model="gpt-4o-mini"):
    """
    뉴스 요약 파싱 정보를 DataFrame으로 변환

    Args:
        df (pd.DataFrame): 원본 뉴스 데이터프레임. 최소 컬럼 'id', 'title', 'contents', 'publish_date' 포함.
        cache_file (str, optional): 요약 결과를 저장/로드할 캐시 파일명. Defaults to "cache_output.csv".

    Returns:
        pd.DataFrame: 뉴스 요약 및 사건 정보가 포함된 DataFrame. 
                      컬럼:
                        - 'id': 뉴스 ID
                        - 'summary': 주요 사건 요약 (문자열)
                        - 'keywords': 주요 키워드 (리스트)
                        - 'event_title': 사건 제목 (문자열)
                        - 'event_date': 사건 발생일 (yyyy-mm-dd)
                        - 'event_person': 사건 핵심 인물 (리스트)
                        - 'event_org': 사건 핵심 조직/기관 (리스트)
                        - 'event_loc': 사건 발생 지명 (리스트)
                        - 'p_rice(won/kg)': 평양 쌀 가격 (정수)
                        - 'p_corn(won/kg)': 평양 옥수수 가격 (정수)
                        - 'p_usd(won/usd)': 원/달러 환율 (정수)
                        - 'job_cost': 처리 비용 추정 (float)
    
    Example:
        >>> df_summary = get_article_summary(news_df)
        >>> df_summary.head()

    Change Log:
        @@ 11.19
        - 캐싱 기능 추가: 뉴스 id 기반으로 이미 요약된 뉴스는 스킵하도록 수정.
        - 새로운 뉴스가 추가될 경우에만 LLM 호출 수행.
        - 쉼표로 구분된 항목(event_person, event_org, event_loc, keywords)을 리스트 형태로 변환.
        @@ 11.20
        - content 줄바꿈, 공백 문자 공백(' ')으로 처리 (실패, 원하는 결과를 얻지 못함)
        - [id 조회 --> df 필터 -->> 판다스 시리즈로 전달] 방식으로 수정 (정보가 제대로 추출되지 않는듯함.)
            > 해당 방식으로 진행시 아무런 가격 정보가 추출되지 않았다.
        - 다시 원래 content 줄바꿈, 공백 문자 공백(' ')으로 처리 방식으로 돌아옴.
        - "gpt-4.1-nano" 테스트 -> 가격 혼동 문제는 해결
        - 함수에 model 선택 파라미터 추가
    """
    
########## 코드 시작 ##########
    # ---------------------------------------------------------------------------------
    # 0. 기본 설정 / 스키마 정의
    # ---------------------------------------------------------------------------------
    new_cols = [
        "summary", "keywords", "event_title", "event_date", "event_person",
        "event_org", "event_loc", "p_rice(won/kg)", "p_corn(won/kg)",
        "p_usd(won/usd)", #"job_cost"
    ]

    cache_dir = "data/cached"
    os.makedirs(cache_dir, exist_ok=True)
    cache_file_path = os.path.join(cache_dir, cache_file)

    # 스키마 정의 (dtype 안정화)
    schema = {
        "id": pd.Series(dtype="object"),
        "summary": pd.Series(dtype="object"),
        "keywords": pd.Series(dtype="object"),
        "event_title": pd.Series(dtype="object"),
        "event_date": pd.Series(dtype="object"),
        "event_person": pd.Series(dtype="object"),
        "event_org": pd.Series(dtype="object"),
        "event_loc": pd.Series(dtype="object"),
        "p_rice(won/kg)": pd.Series(dtype="Int64"),
        "p_corn(won/kg)": pd.Series(dtype="Int64"),
        "p_usd(won/usd)": pd.Series(dtype="Int64"),
        # "job_cost": pd.Series(dtype="float"),
    }

    # ---------------------------------------------------------------------------------
    # 1. 캐시 로드 or 초기화
    # ---------------------------------------------------------------------------------
    if os.path.exists(cache_file_path):
        print("⏳ 캐시 파일 로드 중...")
        output_df = pd.read_csv(cache_file_path)

        # schema 기준으로 dtype 강제 적용
        for col, series in schema.items():
            if col not in output_df.columns:
                output_df[col] = series  # 없는 컬럼은 빈 Series 생성
            else:
                output_df[col] = output_df[col].astype(series.dtype, errors="ignore")

        print("✔ 캐시 반영 완료!")
    else:
        print("캐시 없음 → 빈 DF 생성")
        output_df = pd.DataFrame(schema)

    # ---------------------------------------------------------------------------------
    # 2. 신규 기사만 처리
    # ---------------------------------------------------------------------------------
    cached_ids = set(output_df["id"].astype(str).tolist())
    new_articles = df[~df["id"].astype(str).isin(cached_ids)]

    print(f"신규 샘플 발견: {len(new_articles)}개")

    # ---------------------------------------------------------------------------------
    # 3. 루프 처리 / row 1개씩 추가 
    # ---------------------------------------------------------------------------------
    for cnt, (i, row) in enumerate(new_articles.iterrows()): 
        article_id = row['id']
        title = row['title']
        contents = row['contents']
        publish_date = row['publish_date']

        print(f"Summarizing [{article_id}] ...")

        # LLM 호출
        result, input_tokens, output_tokens = get_article_summary(title, contents, publish_date, model)

        # 새 row 구성 (None → pd.NA)
        new_data = {
            "id": article_id,
            "summary": result.get("summary"),
            "keywords": value_to_strCSV(result.get("keywords")),
            "event_title": result.get("event_title"),
            "event_date": result.get("event_date"),
            "event_person": value_to_strCSV(result.get("event_person")),
            "event_org": value_to_strCSV(result.get("event_org")),
            "event_loc": value_to_strCSV(result.get("event_loc")),
            "p_rice(won/kg)": pd.NA if result.get("p_rice(won/kg)")==0 or result.get("p_rice(won/kg)") is None else result.get("p_rice(won/kg)"),
            "p_corn(won/kg)": pd.NA if result.get("p_rice(won/kg)")==0 or result.get("p_corn(won/kg)") is None else result.get("p_corn(won/kg)"),
            "p_usd(won/usd)": pd.NA if result.get("p_usd(won/usd)")==0 or result.get("p_usd(won/usd)") is None else result.get("p_usd(won/usd)"),
            "job_cost": (
                input_tokens * 0.15 / 1_000_000 +
                output_tokens * 0.60 / 1_000_000
            ),
        }

        # 컬럼 순서와 dtype 맞춰서 row 추가
        new_data_aligned = {col: new_data.get(col, pd.NA) for col in output_df.columns}
        
        new_row_df = pd.DataFrame([new_data_aligned]).dropna(axis=1, how='all')  # dict → DataFrame 변환, na값이 있는 컬럼 제거
        output_df = pd.concat([output_df, new_row_df], ignore_index=True)

        # 캐시 저장
        output_df.to_csv(cache_file_path, index=False)

        print(f"Job Complete! [{article_id}] ({cnt+1}/{len(new_articles)})")

    print("전체 작업 완료!")
    return output_df

### **[LLM > DB 함수]**

In [3]:
class LLMtoDatabase:

    def __init__(self, host, database, user, password, port, tfidf_vectorizer_path, svm_model_path, label_encoder_path):
        """

        CSV(title, contents, publish_date, url) 파일을 받아 LLM 요약.
        LLM ouput Postgre DB table에 저장.

        [수정 2025-11-27]
        1) CSV(title, contents, publish_date, url) 파일을 input으로 받음.
        2) LLM summary, keywords 등등 출력.
        3) summary, keywords TF-IDF 변환 > SVM > 카테고리 분류.
        4) ADA embedding 진행 > db 저장. (postgres에서 l2 norm 진행)
        5) Postgre DB table에 저장.

        [수정 2025-11-28]
        매개변수 tfidf_vectorizer_path, svm_model_path, label_encoder_path 추가.
        '시', '도' 변환 코드 merge.

        """

        # Postgre db 연결
        self.conn = psycopg2.connect(host=host, database=database, user=user, password=password, port=port)
        self.cur = self.conn.cursor()

        # TF-IDF pickle 파일 로드
        with open(tfidf_vectorizer_path, "rb") as f:
            self.tfidf_vectorizer = pickle.load(f)

        with open(svm_model_path, "rb") as f:
            self.svm_model = pickle.load(f)

        with open(label_encoder_path, "rb") as f:
            self.label_encoder = pickle.load(f) 

        # Load nk_cities and build maps for normalization
        try:
            self.nk_cities = pd.read_csv('data/nk_cities.csv', encoding='euc-kr')
            self.provinces_map, self.cities_map = self._build_maps()
            self.BROAD_TERMS_MAP = {
                "평안도": ["평안남도", "평안북도"],
                "함경도": ["함경남도", "함경북도"],
                "황해도": ["황해남도", "황해북도"]
            }
        except Exception as e:
            print(f"Warning: Failed to load nk_cities.csv or build maps. Normalization will be skipped. Error: {e}")
            self.nk_cities = None
            self.provinces_map = {}
            self.cities_map = {}
            self.BROAD_TERMS_MAP = {}      

    def _get_search_keys(self, name):
        """

        [추가 2025-11-28]


        """
        if pd.isna(name): return [], None
        # Handle parentheses: "나선시(라선시)" -> parts: ["나선시", "라선시"]
        parts = re.split(r'[()]', name)
        parts = [p.strip() for p in parts if p.strip()]
        
        canonical_name = parts[0] # The first part is the canonical name
        
        keys = []
        for p in parts:
            # Strip suffixes '도', '시', '군', '구역' for search key
            key = p
            if key.endswith('도'): key = key[:-1]
            elif key.endswith('시'): key = key[:-1]
            elif key.endswith('군'): key = key[:-1]
            elif key.endswith('구역'): key = key[:-1]
            keys.append(key)
        return keys, canonical_name

    def _build_maps(self):
        """

        [추가 2025-11-28]
        1) 북한 행정구역 정리 파일(nk_cities.csv) 
        2) 약어 혹은 확장명 매핑

        """
        provinces_map = {} # search_key -> canonical_full_name
        cities_map = {}    # search_key -> {'full': canonical_full_name, 'province': province_canonical_name}

        for idx, row in self.nk_cities.iterrows():
            # Process Province
            p_keys, p_canon = self._get_search_keys(row['도'])
            for k in p_keys:
                provinces_map[k] = p_canon
                
            # Process City
            c_keys, c_canon = self._get_search_keys(row['시'])
            for k in c_keys:
                cities_map[k] = {
                    'full': c_canon,
                    'province': p_canon # This might be None or a string
                }

        # Manual additions for abbreviations and broader terms
        abbr_map = {
            '평남': '평안남도',
            '평북': '평안북도',
            '함남': '함경남도',
            '함북': '함경북도',
            '황남': '황해남도',
            '황북': '황해북도',
            '양강': '양강도',
            '자강': '자강도',
            '강원': '강원도',
            '평안도': '평안도', # Broader term
            '황해도': '황해도', # Broader term
            '함경도': '함경도',  # Broader term
            '평안': '평안도' # Example 7: "평안" -> "평안도" (Assuming broader term)
        }

        for abbr, full in abbr_map.items():
            provinces_map[abbr] = full
            
        return provinces_map, cities_map

    def map_location_normalized(self, loc_str):
        """

        [추가 2025-11-28]
        rule-based 로 관리.
        1) 광역시(남포시, 개성시, 나선시(라선시), 평양시)인 경우 '도' 없어도 표기.
        2) 광역시가 아닌 '시'의 경우, 매핑된 '도'를 함께 표기.
        3) '도' 항상 표기.

        """
        if pd.isna(loc_str) or not isinstance(loc_str, str):
            return None
        
        found_provinces = set()
        found_cities = [] # List of dicts
        
        # 1. Search for Provinces
        for key, full_name in self.provinces_map.items():
            if key in loc_str:
                found_provinces.add(full_name)
                
        # 2. Search for Cities
        for key, info in self.cities_map.items():
            if key in loc_str:
                match_info = info.copy()
                match_info['key'] = key
                found_cities.append(match_info)
                
        # 3. Consolidate and Remove Redundancy
        
        # 3a. Identify implied provinces from found cities
        implied_provinces = set()
        for c in found_cities:
            if pd.notna(c['province']):
                implied_provinces.add(c['province'])
                
        # 3b. Remove found provinces if they are implied by the cities
        temp_provinces = set()
        for p in found_provinces:
            if p not in implied_provinces:
                temp_provinces.add(p)
        
        # 3c. Remove Broad Terms if Specific Terms are present
        all_present_specific_provinces = temp_provinces.union(implied_provinces)
        
        final_provinces = set()
        for p in temp_provinces:
            is_redundant_broad = False
            if p in self.BROAD_TERMS_MAP:
                # Check if any specific term for this broad term is present
                for specific in self.BROAD_TERMS_MAP[p]:
                    if specific in all_present_specific_provinces:
                        is_redundant_broad = True
                        break
            
            if not is_redundant_broad:
                final_provinces.add(p)
                
        # 4. Format Output
        final_results = set()
        
        # Add Remaining Provinces
        for p in final_provinces:
            final_results.add(p)
            
        # Add Cities (Format: "Province City" or "City")
        for c in found_cities:
            full_city = c['full']
            province = c['province']
            
            if pd.notna(province):
                final_results.add(f"{province} {full_city}")
            else:
                final_results.add(full_city)
                
        if not final_results:
            return None
            
        return ', '.join(sorted(list(final_results)))    


    def get_article_summary(self, title, contents, publish_date):
        """
        뉴스 기사를 LLM으로 요약하고 항목별 데이터 반환
        
        Change Log:
            @@ 11.18 
            > 프롬프트 수정
                - 키값 정리 형식 재 설정 - 데이터 프레임의 컬럼명이 키값이 되도록 수정
                - 추가적인 컬럼 event_person 추가
                - event_obj 컬럼명 event_org로 변경
            @@ 11.19
            > 프롬프트 수정
                - 인물명은 이름만 명확하게
                - 지역명은 [국가, 도, 시] 단위로 명확하게
                - 평양 쌀, 옥수수, 달러환율 정보는 각기 별개의 데이터로 저장
                - 키워드는 인물명, 지역명을 반드시 포함 + 요약내용 대표 단어 추가
            > result 파싱 방식 수정
                - eval()에서 jason.load()로 변경
        """
        
        prompt = f"""
    아래 기사를 분석하여 요구된 정보를 작성하시오.

    # 기사 제목:
    {title}

    # 기사 내용:
    {contents}

    # 기사 작성일:
    {publish_date}

   1. 아래 형식으로 정리 (괄호안 각 key값의 한글 설명은 참고만 하고 최종 결과에는 포함하지 않음)
    - summary(주요 사건 요약):
    - event_title(사건 주제):
    - event_date(사건 발생일):
    - event_person(사건 핵심 인물):
    - event_org(사건 핵심 조직/기관):
    - event_loc(사건 발생 지명):
    - keywords(주요 키워드):
    
    2. 각 카테고리의 조건
    - "summary": 3 문장 이하로 핵심 내용만 발췌.
    - "event_title": 간단한 한 문장으로 사건 주제 작성.
    - "event_date": yyyy-mm-dd 형식, 기사에 "event_date"가 명시되지 않았으면 "기사 내용" 중 시간 또는 기간을 나타내는 단어(예시로, '어제', '사흘전', '일주일 전' 등)를 참고하여 "기사 작성일" 기준 계산.
    - "event_person": 사건의 주체 인물(들)의 이름만 입력, 다수의 경우 쉼표로 구분.
    - "event_org": 사건의 주체 조직 및 기관의 이름만 입력, 다수의 경우 쉼표로 구분, **언론사명은 반드시 제외**, **신문사명은 반드시 제외**, **기자가 참고한 출처의 이름도 반드시 제외**, **"노동신문"은 반드시 제외**.
    - "event_loc": [도, 시]단위 지명만을 입력하되 "도" 와 "시" 정보가 함께 있는 경우는 반드시 행적구역별로 분리해서 입력. 건물등에서 일어난 사건의 경우는 해당 장소의 [도, 시] 지명을 입력, 행정구역이 "시"일 경우는 꼭 "시"를 명시 (개성시, 평양시, 고성시 등). 
    특히 "평양" / "평양직할시" / "평양시"와 같이 한 지명에 다양한 표기가 있을경우는 "평양시" ([시 이름] + 시)와 같은 형태로 통일. **"북한" 이라는 단어는 반드시 제외**. 북한이 아닌 해외의 사건의 경우만 국가명을 입력.
    - "keywords": "summary", "event_title", "event_person", "event_org", "event_loc" 모두를 종합적으로 고려하여 해당 뉴스 사건을 대표할 수 있는 **단어 5개 선정**, **"북한" 이라는 단어는 반드시 제외**, 쉼표로 구분하여 입력.
    
    - 위 결과를 종합하여 딕셔너리 형태로 출력.
    - 결과를 출력하기 전 다음 체크리스트를 스스로 검증하라:
        - [ ] 내가 사용한 모든 답과 수치는 기사 원문에 존재한다.
        
    - 설명 출력 금지, 답만 출력.
    """
        response = client.chat.completions.create(
            model=llm_model,
            messages=[
                {"role": "system", "content": "당신은 북한 관련 뉴스 사건 정보를 추출하는 전문 분석 모델입니다."},
                {"role": "user", "content": prompt},
            ],
            temperature=0
        )
        
        # 문자열을 dict로 변환
        result_text = response.choices[0].message.content.strip()
        try:
            result = json.loads(result_text)  # json 파싱으로 수정(11.19)
        except:
            print("Parsing error:", result_text)
            return None
              
        return result
        
    def value_to_strCSV(self, value):
        """
        리스트 또는 쉼표 문자열을 'a, b, c' 형태의 문자열로 표준화
        """
        if not value:
            return ""

        # 이미 리스트이면 → 요소 strip 후 join
        if isinstance(value, list):
            return ", ".join(x.strip() for x in value)

        # 문자열이면 split → 다시 join 처리
        if isinstance(value, str):
            return ", ".join(x.strip() for x in value.split(","))

        # 그 외 타입은 문자열로 강제 변환
        return str(value)
    
    def insert_summary(self, llm, title, publish_date, url, category, embedding):
        """
        
        LLM output 과 원본 csv 파일의 title, publish_date, url 데이터를 postgre table에 저장

        [수정 2025-11-24]
        issue: 동일 csv 파일로 코드 재실행 시, 이미 db에 등록된 contents가 새로운 id로 재등록.
        수정: 동일한 url이 재입력 시, pass 
        상세: query >> ON CONFLICT (url) DO NOTHING << 추가

        """

        query = """
            INSERT INTO summary
                (summary, keywords, event_title, event_date,
                 event_person, event_org, event_loc, url, title, publish_date, category, embedding)
            VALUES
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (url) DO NOTHING;
        """

        values = (
            llm.get("summary"),
            self.value_to_strCSV(llm.get("keywords")),
            llm.get("event_title"),
            llm.get("event_date"),
            self.value_to_strCSV(llm.get("event_person")),
            self.value_to_strCSV(llm.get("event_org")),
            self.value_to_strCSV(llm.get("event_loc")),
            url,
            title, 
            publish_date,
            category,
            embedding.tolist(),
        )

        try:
            self.cur.execute(query, values)
            self.conn.commit()

            if self.cur.rowcount == 0:
                print(f"[DB INSERT ERROR] 이미 존재하는 기사입니다. url={url}")

        except Exception as e:
            self.conn.rollback()
            print(f"[DB INSERT ERROR] url={url} ⇒ {e}")

    def check_url(self, url):
        """

        [추가 2025-11-24]
        issue: LLM 처리 후 url 중복 체크 시, LLM 토큰 낭비.
        해결: LLM 처리 이전 url 사전 체크 함수 추가. 
        상세: 
        코드 재실행 시 동일한 기사가 새로운 ID를 부여받지 않도록 방지.
        url을 조회하여 있는 경우 skip.

        """
        query = """
            SELECT COUNT(*) FROM summary
            WHERE url = %s;
        """

        self.cur.execute(query, (url,))
        count = self.cur.fetchone()[0]

        return count > 0
    
    def preprocess_text(self, text):
        """

        [추가 2025-11-27]
        issue: SVM 모델 구동 위해 형태 동일하게 변환.
        해결: s_news_categorizer 전처리 코드와 동일한 로직.

        """
        if pd.isna(text): 
            return ""
        text = str(text).lower() 
        text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text) 
        return text
    
    def get_category(self, summary, keywords):
        """

        [추가 2025-11-27]
        issue: 모델 분류기가 tf-idf를 활용함에 따라 tf-idf 변환 필요.
        해결: s_news_categorizer 전처리 코드 merge.
        상세: s_news_categorizer.ipynb 동일. 
        
        [수정 2025-11-28]
        summary + keywords 를 input을 받는 vectorizer 로 변경.
        vectorizer >> svm >> label encoding

        """
        preprocessed_summary = self.preprocess_text(summary)
        preprocessed_keywords = self.preprocess_text(keywords)
        combined_text = preprocessed_summary + " " + preprocessed_keywords

        X_combined = self.tfidf_vectorizer.transform([combined_text])

        svm_pred = self.svm_model.predict(X_combined)[0]

        category = self.label_encoder.inverse_transform([svm_pred])[0]
        return category
    
    def text_to_embedding(self, text):
        """

        [추가 2025-11-27]
        issue: 모델 분류기는 tf-idf를 활용하지만, 추천시스템은 embedding 활용.
        해결: embedding 코드 추가.
        상세: summary + keywords 를 임베딩 후 수평합. 해당 값은 postgres에 저장.

        [수정 2025-11-28]
        issue: text_embedding 라이브러리와 파이썬 버전 충돌
        해결: text_embeddings[0].embedding >> text_embeddings.data[0].embedding

        """
        text = text
        text_embeddings = client.embeddings.create(
            model = embed_model,
            input = text
        )
        embeddings = np.array(text_embeddings.data[0].embedding, dtype = np.float32)
        return embeddings
    
    def get_embeddings(self, summary, keywords):
        """

        [추가 2025-11-27]
        issue: 모델 분류기는 tf-idf를 활용하지만, 추천시스템은 embedding 활용.
        해결: embedding 코드 추가.
        상세: text_to_embedding 함수 받아와 summary, keywords 임베딩.

        """
        embed_summary = self.text_to_embedding(summary)
        embed_keywords = self.text_to_embedding(keywords)
        embed_rec = np.hstack([embed_summary, embed_keywords])
        return embed_rec

    
    def close(self):
        self.cur.close()
        self.conn.close()

In [ ]:
# 실행 코드

df = pd.read_csv("data/test_region_parsing.csv", encoding="cp949")  
# 반드시 포함: title, contents, publish_date, url

llm_db = LLMtoDatabase(
    host="localhost",
    database="nvisiaDb",
    user="postgres",
    password="postgres1202",
    port=5432,
    tfidf_vectorizer_path = "pickle/s_news_cate_tfidf_xcombined_vec.pkl", 
    svm_model_path = "pickle/s_news_cate_model.pkl", 
    label_encoder_path = "pickle/s_news_cate_label_en.pkl"
)

for idx, row in df.iterrows():
    title = str(row["title"])
    contents = str(row["contents"])
    publish_date = str(row["publish_date"])
    url = str(row["url"])

    if llm_db.check_url(url):
        print(f"[중복] 이미 존재하는 기사입니다. 이어서 다음 기사를 분석합니다.")
        continue

    # LLM 
    llm_output = llm_db.get_article_summary(title, contents, publish_date)

    if llm_output is None:
        print(f"[에러] 데이터가 누락되어 다음 행으로 넘어갑니다. {idx}")
        continue

    # event_loc 정규화
    raw_loc = llm_output.get("event_loc")
    norm_loc = llm_db.map_location_normalized(raw_loc)
    llm_output["event_loc"] = norm_loc 

    # category 분류
    summary_text = llm_output.get("summary")
    keywords_text = llm_output.get("keywords")
    category = llm_db.get_category(summary_text, keywords_text)

    # embedding
    embedding = llm_db.get_embeddings(summary_text, keywords_text)

    # DB 저장
    llm_db.insert_summary(llm_output, title, publish_date, url, category, embedding)

    print(f"[저장] 행 업로드 되었습니다. {idx}")

llm_db.close()
print("[종료] 모든 업로드가 완료되었습니다.")

c:\Users\user\Desktop\MODULABS\Aiffelthon\project_NVSIA\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\user\Desktop\MODULABS\Aiffelthon\project_NVSIA\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\user\Desktop\MODULABS\Aiffelthon\project_NVSIA\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Tr

[저장] 행 업로드 되었습니다. 0
[저장] 행 업로드 되었습니다. 1
[저장] 행 업로드 되었습니다. 2
[저장] 행 업로드 되었습니다. 3
[저장] 행 업로드 되었습니다. 4
[저장] 행 업로드 되었습니다. 5
[저장] 행 업로드 되었습니다. 6
[저장] 행 업로드 되었습니다. 7
[저장] 행 업로드 되었습니다. 8
[저장] 행 업로드 되었습니다. 9
[저장] 행 업로드 되었습니다. 10
[저장] 행 업로드 되었습니다. 11
[저장] 행 업로드 되었습니다. 12
[저장] 행 업로드 되었습니다. 13
[저장] 행 업로드 되었습니다. 14
[저장] 행 업로드 되었습니다. 15
[저장] 행 업로드 되었습니다. 16
[저장] 행 업로드 되었습니다. 17
[저장] 행 업로드 되었습니다. 18
[저장] 행 업로드 되었습니다. 19
[저장] 행 업로드 되었습니다. 20
[저장] 행 업로드 되었습니다. 21
[저장] 행 업로드 되었습니다. 22
[저장] 행 업로드 되었습니다. 23
[저장] 행 업로드 되었습니다. 24
[종료] 모든 업로드가 완료되었습니다.


In [19]:
# llm_db.close()